In [226]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np

In [227]:
res = requests.get('https://etfdb.com/history-of-the-s-and-p-500/#2009')
soup = bsoup(res.content,'html.parser')

In [228]:
def extractTable(soup):
    '''Extract the top ten companies from 1980-2013 '''
    table = []
    for div in soup.find_all('div', {'id':'sections'}):
        for a in div.find_all('div', {'class':'dataRow'}):
            for s in a.find_all('span', {'class': 'rowTitle'}):
                table+=s
    return(table)

In [229]:
tbl = extractTable(soup)
# Remove companies 1980-1999
Lst = tbl[200:]

def getComp(Lst):
    return [Lst[i::10] for i in range(10)]
# Yield successive n-sized 
def divide_chunks(Lst, n): 

    for i in range(0, len(Lst), n):  
        yield Lst[i:i + n] 
        


In [256]:
# Split into chunks of 10 CO per year 
tbl = list(divide_chunks(Lst, 10))
colNames=[i for i in range(2000,2014)]

In [259]:
# Make df
dfTop10 = pd.DataFrame(tbl).transpose()
dfTop10.columns = colNames

In [260]:
# Remove punctuation and uppercase 
for i in colNames:
    dfTop10[i] = dfTop10[i].str.upper().str.strip(string.punctuation).str.strip('\n')


In [261]:
dfTop10.replace({'MICROSOFT': 'MICROSOFT CORP','GENERAL ELECTRIC': 'GENERAL ELECTRIC CO', 'EXXON MOBIL':'EXXON MOBIL CORP', 'PFIZER':'PFIZER INC', 'CITIGROUP':'CITIGROUP INC' }, regex=True, inplace=True)

In [265]:
dfCik = pd.read_csv('2000_2013_s&p.csv')
dfCik.drop(['gvkey','gvkeyx','from','thru','tic'], axis=1, inplace=True)
dfCik['co_cik'] = dfCik['co_cik'].fillna(0).astype(int).apply(lambda x: '{0:0>7}'.format(x))
cikDic = pd.Series(dfCik.co_conm.values,index=dfCik.co_cik).to_dict()
cikDic = dict((v,k) for k,v in cikDic.items())

In [266]:
cikDic

{'ABBOTT LABORATORIES': '0001800',
 'HONEYWELL INTERNATIONAL INC': '0773840',
 'AMERICAN ELECTRIC POWER CO': '0004904',
 'BOEING CO': '0012927',
 'BRISTOL-MYERS SQUIBB CO': '0014272',
 'CAMPBELL SOUP CO': '0016732',
 'CATERPILLAR INC': '0018230',
 'CHEVRON CORP': '0093410',
 'COCA-COLA CO': '0021344',
 'COLGATE-PALMOLIVE CO': '0021665',
 'CONSOLIDATED EDISON INC': '1047862',
 'CORNING INC': '0024741',
 'DEERE & CO': '0315189',
 'DTE ENERGY CO': '0936340',
 'DOMINION ENERGY INC': '0715957',
 'DOWDUPONT INC': '1666700',
 'EATON CORP PLC': '1551182',
 'EXXON MOBIL CORP': '0034088',
 'FORD MOTOR CO': '0037996',
 'GENERAL DYNAMICS CORP': '0040533',
 'GENERAL ELECTRIC CO': '0040545',
 'GOODYEAR TIRE & RUBBER CO': '0042582',
 'HALLIBURTON CO': '0045012',
 'HERSHEY CO': '0047111',
 'INTL BUSINESS MACHINES CORP': '0051143',
 'INTL PAPER CO': '0051434',
 'KELLOGG CO': '0055067',
 'KIMBERLY-CLARK CORP': '0055785',
 'KROGER CO': '0056873',
 'S&P GLOBAL INC': '0064040',
 'CVS HEALTH CORP': '0064803

In [267]:
dfTop10

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,MICROSOFT CORP,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,APPLE,APPLE
1,EXXON MOBIL CORP,MICROSOFT CORP,GENERAL ELECTRIC CO,MICROSOFT CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,GENERAL ELECTRIC CO,GENERAL ELECTRIC CO,WAL-MART STORES,MICROSOFT CORP,APPLE,APPLE,EXXON MOBIL CORP,EXXON MOBIL CORP
2,PFIZER INC,EXXON MOBIL CORP,EXXON MOBIL CORP,EXXON MOBIL CORP,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,PROCTER & GAMBLE,WAL-MART STORES,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP,GOOGLE
3,CITIGROUP INC,CITIGROUP INC,WAL-MART STORES,PFIZER INC,CITIGROUP INC,CITIGROUP INC,CITIGROUP INC,AT&T,MICROSOFT CORP,GOOGLE,BERKSHIRE HATHAWAY,IBM,GENERAL ELECTRIC CO,MICROSOFT CORP
4,CISCO SYSTEMS,WAL-MART STORES,PFIZER INC,CITIGROUP INC,WAL-MART STORES,PROCTER & GAMBLE,BANK OF AMERICA,PROCTER & GAMBLE,AT&T,APPLE,GENERAL ELECTRIC CO,CHEVRON,IBM,GENERAL ELECTRIC CO
5,WAL-MART STORES,PFIZER INC,CITIGROUP INC,WAL-MART STORES,PFIZER INC,WAL-MART STORES,PROCTER & GAMBLE,GOOGLE,JOHNSON & JOHNSON,JOHNSON & JOHNSON,WAL-MART STORES,GOOGLE,CHEVRON,JOHNSON & JOHNSON
6,MICROSOFT CORP,INTEL,JOHNSON & JOHNSON,INTEL,BANK OF AMERICA,BANK OF AMERICA,WAL-MART STORES,CHEVRON,GENERAL ELECTRIC CO,PROCTER & GAMBLE,GOOGLE,WAL-MART STORES,JOHNSON & JOHNSON,CHEVRON
7,AIG,IBM,AIG,AIG,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,JOHNSON & JOHNSON,CHEVRON CORP,IBM,CHEVRON CORP,GENERAL ELECTRIC CO,AT&T,PROCTER & GAMBLE
8,MERCK,AIG,IBM,CISCO SYSTEMS,AIG,AIG,PFIZER INC,WAL-MART STORES,PFIZER INC,AT&T,IBM,BERKSHIRE HATHAWAY,PROCTER & GAMBLE,JPMORGAN CHASE
9,INTEL,JOHNSON & JOHNSON,MERCK,IBM,IBM,PFIZER INC,AIG,BANK OF AMERICA,JPMORGAN CHASE,JPMORGAN CHASE,PROCTER & GAMBLE,PROCTER & GAMBLE,PFIZER INC,WELLS FARGO


In [251]:
for i in dfTop10[2000]:
    ID = []
    #print(i)
    if i in cikDic:
        ID += cikDic[i]
    else:
        ID.append(None)

In [248]:
cikDic['GENERAL ELECTRIC CO']

'0040545'